# 動態提示鏈 (Dynamic Prompt Chains) - 推薦最佳實踐 ⭐

本範例展示如何讓 RunnableLambda 動態準備 Prompt，然後串接模型調用。這是處理複雜邏輯的推薦方法。

## 🎯 學習目標
- 理解動態 Prompt 準備的概念
- 掌握根據內容動態調整處理邏輯的技巧
- 學習結合靈活性和可讀性的最佳實踐

## 📝 核心概念
動態提示鏈讓 RunnableLambda 只負責準備和調整 Prompt 參數，然後串接標準的 Prompt Template 和模型，兼具靈活性和可維護性。

## ✨ 適用場景
- 需要根據內容動態調整處理邏輯
- 智能路由增強版
- 自適應處理流程
- 複雜條件下的 Prompt 準備

## 🌟 為什麼這是最佳實踐？
1. **結構清晰**：鏈的流程一目了然
2. **易於維護**：Prompt 和處理邏輯分離
3. **高度靈活**：可以根據內容動態調整
4. **便於調試**：每個步驟都可以單獨測試

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_ollama.llms import OllamaLLM

load_dotenv()
model = OllamaLLM(model="llama3.2:latest")
print("✅ 模型已建立")

In [ ]:
# 主提示模板
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是專業的客服代表。"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

print("✅ 主提示模板已建立")

## 核心：動態 Prompt 準備函數

這個函數會：
1. 分析回覆內容
2. 格式化郵件
3. 根據內容類型決定檢查重點
4. 返回 dict 格式的數據給下一個 Prompt

In [ ]:
def prepare_quality_check_prompt(reply):
    """
    根據回覆內容動態準備品質檢查 Prompt
    
    關鍵點：
    - 只負責準備數據，不直接調用模型
    - 返回 dict 格式，傳給下一個 Prompt Template
    - 根據內容動態調整檢查重點
    """
    
    # 步驟 1：格式化郵件
    formatted = f"""
親愛的客戶，

{reply}

此致
客服團隊
"""
    
    # 步驟 2：智能分析內容，決定檢查重點
    if "退貨" in reply or "換貨" in reply:
        focus = "請特別注意退換貨政策的說明是否清楚、完整。評估是否明確告知客戶處理流程。"
    elif "抱歉" in reply or "歉意" in reply:
        focus = "請特別注意道歉的誠意和補償措施。評估是否展現足夠的同理心和解決問題的決心。"
    elif "感謝" in reply:
        focus = "請評估回覆的友善度和專業度。確認是否適當表達了對客戶的重視。"
    else:
        focus = "請評估整體的專業度、友善度和完整度。"
    
    # 步驟 3：返回 dict 格式，供下一個 Prompt 使用
    return {
        "email": formatted,
        "focus": focus
    }

print("✅ 動態 Prompt 準備函數已定義")

In [ ]:
# 品質檢查 Prompt（接收動態準備的數據）
dynamic_quality_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是郵件品質審查專家。{focus}"),
    ("human", "評估這封郵件：\n\n{email}")
])

# 建立完整的動態提示鏈
dynamic_chain = (
    prompt_template                             # 1. 主 Prompt
    | model                                     # 2. 主模型生成回覆
    | StrOutputParser()                         # 3. 解析為字串
    | RunnableLambda(prepare_quality_check_prompt)  # 4. 動態準備檢查 Prompt
    | dynamic_quality_prompt                    # 5. 品質檢查 Prompt
    | model                                     # 6. 品質檢查模型
    | StrOutputParser()                         # 7. 解析結果
)

print("✅ 動態提示鏈已建立完成！")
print("\n流程：主 Prompt → 主模型 → 動態準備檢查 Prompt → 品質檢查模型")
print("\n✨ 特點：Lambda 只負責準備數據，模型調用清晰可見")

In [ ]:
# 測試案例 1：退換貨問題
customer_email_1 = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

print("="*60)
print("測試案例 1：退換貨問題（應強調退換貨政策）")
print("="*60)
result1 = dynamic_chain.invoke({"email_content": customer_email_1})
print(result1)

In [ ]:
# 測試案例 2：投訴問題
customer_email_2 = """
你們的服務態度太差了！
我要投訴！
"""

print("\n" + "="*60)
print("測試案例 2：投訴問題（應強調道歉誠意）")
print("="*60)
result2 = dynamic_chain.invoke({"email_content": customer_email_2})
print(result2)

In [ ]:
# 測試案例 3：一般諮詢
customer_email_3 = """
請問你們的營業時間？
"""

print("\n" + "="*60)
print("測試案例 3：一般諮詢（評估整體專業度）")
print("="*60)
result3 = dynamic_chain.invoke({"email_content": customer_email_3})
print(result3)

## 📊 動態提示鏈總結

### ✅ 優點（最佳實踐）
- **結構最清晰**：鏈的流程一目了然，易於理解
- **高度靈活**：可以根據內容動態調整處理邏輯
- **易於維護**：Prompt 和準備邏輯分離
- **便於調試**：每個步驟都清晰可見，易於測試
- **可擴展性強**：容易添加新的動態邏輯

### 🎯 與其他方法的比較

| 方法 | 可讀性 | 靈活性 | 維護性 | 推薦度 |
|------|--------|--------|--------|--------|
| **動態提示鏈（本方法）** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | 🏆 最推薦 |
| 閉包模型鏈 | ⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐ | 謹慎使用 |
| 並行模型鏈 | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | 特定場景 |
| 串聯模型鏈 | ⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐ | 簡單場景 |

### 🎯 最佳實踐建議

1. **優先使用動態提示鏈**：當需要根據內容調整邏輯時
2. **Lambda 只做數據準備**：不在 Lambda 中直接調用模型
3. **返回 dict 格式**：方便 Prompt Template 使用
4. **添加清晰註解**：說明每個動態決策的邏輯
5. **保持鏈的可見性**：讓模型調用清晰呈現在鏈結構中

### 🔗 相關技術
- [串聯模型鏈](5_chains_sequential_model_ollama.ipynb)：簡單的順序調用
- [閉包模型鏈](7_chains_closure_model_ollama.ipynb)：完全控制但可讀性較差
- [並行模型鏈](8_chains_parallel_model_ollama.ipynb)：並行處理多個任務
- [Lambda 模型整合](10_chains_lambda_integration_ollama.ipynb)：綜合進階範例